

下面我们实现*k*均值算法，进行文本聚类。这里使用的数据集与第4章的数据集类似，包含3种主题约1万本图书的信息，但文本内容是图书摘要而非标题。首先我们复用第4章的代码进行预处理。


In [2]:
import os
import sys

# 导入前面实现的Books数据集
sys.path.append('./code')
from utils import BooksDataset

dataset = BooksDataset()
# 打印出类和标签ID
print(dataset.id2label)

dataset.tokenize(attr='abstract')
dataset.build_vocab(min_freq=3)
dataset.convert_tokens_to_ids()

train_data, test_data = dataset.train_data, dataset.test_data

train size = 8627 , test size = 2157
{0: '计算机类', 1: '艺术传媒类', 2: '经管类'}


100%|██████████| 2157/2157 [00:47<00:00, 45.56it/s]


unique tokens = 34252, total counts = 806900, max freq = 19197, min freq = 1
min_freq = 3, min_len = 2, max_size = None, remaining tokens = 9504,
 in-vocab rate = 0.8910459784359895


接下来导入实现TF-IDF算法的函数，将处理后的数据集输入到函数中，得到文档特征：

In [3]:
# 导入之前实现的TF-IDF算法
from utils import TFIDF

vocab_size = len(dataset.token2id)
train_X = []
for data in train_data:
    train_X.append(data['token_ids'])
# 对TF-IDF的结果进行归一化（norm='l2'）对聚类非常重要，
# 不经过归一化会导致数据在某些方向上过于分散从而聚类失败
# 初始化TFIDF()函数
tfidf = TFIDF(vocab_size, norm='l2', smooth_idf=True, sublinear_tf=True)
# 计算词频率和逆文档频率
tfidf.fit(train_X)
# 转化为TF-IDF向量
train_F = tfidf.transform(train_X)
print(train_F.shape)

(8627, 9504)


在有了数据之后，运行*k*均值聚类算法为文本进行聚类。我们需要事先确定簇数$K$。为了方便与实际的标签数据进行对比，这里假设$K$为3。

In [10]:
import numpy as np

# 更改簇的标签数量
K = 3

class KMeans:
    def __init__(self, K, dim, stop_val = 1e-4, max_step = 100):
        self.K = K
        self.dim = dim
        self.stop_val = stop_val
        self.max_step = max_step

    def update_mean_vec(self, X):
        mean_vec = np.zeros([self.K, self.dim])
        for k in range(self.K):
            data = X[self.cluster_num == k]
            if len(data) > 0:
                mean_vec[k] = data.mean(axis=0)
        return mean_vec
    
    # 运行k均值算法的迭代循环
    def fit(self, X):
        print('-----------初始化-----------')
        N = len(X)
        dim = len(X[0])
        # 给每个数据点随机分配簇
        self.cluster_num = np.random.randint(0, self.K, N)
        self.mean_vec = self.update_mean_vec(X)
        
        print('-----------初始化完成-----------')
        global_step = 0
        while global_step < self.max_step:
            global_step += 1
            self.cluster_num = np.zeros(N, int) 
            for i, data_point in enumerate(X):
                # 计算每个数据点和每个簇中心的L2距离
                dist = np.linalg.norm(data_point[None, :] - \
                    self.mean_vec, ord=2, axis=-1)
                # 找到每个数据点所属新的聚类
                self.cluster_num[i] = dist.argmin(-1)

            '''
            上面的循环过程也可以以下面的代码进行并行处理，但是可能
            会使得显存过大，建议在数据点的特征向量维度较小时
            或者进行降维后使用
            # N x D - K x D -> N x K x D
            dist = np.linalg.norm(train_X[:,None,:] - self.mean_vec, \
                ord = 2, axis = -1) 
            # 找到每个数据点所属新的聚类
            self.cluster_num = dist.argmin(-1)
            '''

            new_mean_vec = self.update_mean_vec(X)

            # 计算新的簇中心点和上一步迭代的中心点的距离
            moving_dist = np.linalg.norm(new_mean_vec - self.mean_vec,\
                ord = 2, axis = -1).mean()
            print(f"第{global_step}步，中心点平均移动距离：{moving_dist}")
            if moving_dist < self.stop_val:
                print("中心点不再移动，退出程序")
                break

            # 将mean_vec更新
            self.mean_vec = new_mean_vec

kmeans = KMeans(K, train_F.shape[1])
kmeans.fit(train_F)

-----------初始化-----------
-----------初始化完成-----------
第1步，中心点平均移动距离：0.059189038070756865
...
第10步，中心点平均移动距离：0.002389605545132419
...
第16步，中心点平均移动距离：0.0
中心点不再移动，退出程序


为了更直观地展示聚类的效果，我们定义show_clusters()这个函数，显示每个真实分类下包含的每个簇的比重。下面对*k*均值算法的聚类结果进行展示，并观察3个标签中不同簇的占比。

In [11]:
# 取出每条数据的标签和标签ID
labels = []
for data in train_data:
    labels.append(data['label'])
print(len(labels))

# 展示聚类结果
def show_clusters(clusters, K):
    # 每个标签下的数据可能被聚类到不同的簇，因此对所有标签、所有簇进行初始化
    label_clusters = {label_id: {} for label_id in dataset.id2label}
    for k, v in label_clusters.items():
        label_clusters[k] = {i: 0 for i in range(K)}
    # 统计每个标签下，分到每个簇的数据条数
    for label_id, cluster_id in zip(labels, clusters):
        label_clusters[label_id][cluster_id] += 1
        
    for label_id in sorted(dataset.id2label.keys()):
        _str = dataset.id2label[label_id] + ':\t{ '
        for cluster_id in range(K):
            # 计算label_id这个标签ID下，簇为cluster_id的占比
            _cnt = label_clusters[label_id][cluster_id]
            _total = sum(label_clusters[label_id].values())
            _str += f'{str(cluster_id)}: {_cnt}({_cnt / _total:.2f}), '
        _str += '}'
        print(_str)

clusters = kmeans.cluster_num
show_clusters(clusters, K)

8627
计算机类:	{ 0: 2583(0.67), 1: 1222(0.32), 2: 37(0.01), }
艺术传媒类:	{ 0: 281(0.12), 1: 72(0.03), 2: 1947(0.85), }
经管类:	{ 0: 2452(0.99), 1: 26(0.01), 2: 7(0.00), }


接下来演示如何使用高斯混合来进行聚类。注意高斯混合会计算每个数据点归属于各簇的概率分布，这里将概率最高的簇作为聚类输出。

In [12]:
from scipy.stats import multivariate_normal as gaussian
from tqdm import tqdm

# 高斯混合模型
class GMM:
    def __init__(self, K, dim, max_iter=100):
        # K为聚类数目，dim为向量维度，max_iter为最大迭代次数
        self.K = K
        self.dim = dim
        self.max_iter = max_iter
        
        # 初始化，pi = 1/K为先验概率，miu ~[-1,1]为高斯分布的均值，
        # sigma = eye为高斯分布的协方差矩阵
        self.pi = np.ones(K) / K
        self.miu = np.random.rand(K, dim) * 2 - 1
        self.sigma = np.zeros((K, dim, dim))
        for i in range(K):
            self.sigma[i] = np.eye(dim)
        
    # GMM的E步骤
    def E_step(self, X):
        # 计算每个数据点被分到不同簇的密度
        for i in range(self.K):
            self.Y[:, i] = self.pi[i] * gaussian.pdf(X, \
                mean=self.miu[i], cov=self.sigma[i])
        # 对密度进行归一化，得到概率分布
        self.Y /= self.Y.sum(axis=1, keepdims=True)
    
    # GMM的M步骤
    def M_step(self, X):
        # 更新先验概率分布
        Y_sum = self.Y.sum(axis=0)
        self.pi = Y_sum / self.N
        # 更新每个簇的均值
        self.miu = np.matmul(self.Y.T, X) / Y_sum[:, None]
        # 更新每个簇的协方差矩阵
        for i in range(self.K):
            # N * 1 * D
            delta = np.expand_dims(X, axis=1) - self.miu[i]
            # N * D * D
            sigma = np.matmul(delta.transpose(0, 2, 1), delta)
            # D * D
            self.sigma[i] = np.matmul(sigma.transpose(1, 2, 0),\
                self.Y[:, i]) / Y_sum[i]
    
    # 计算对数似然，用于判断迭代终止
    def log_likelihood(self, X):
        ll = 0
        for x in X:
            p = 0
            for i in range(self.K):
                p += self.pi[i] * gaussian.pdf(x, mean=self.miu[i],\
                    cov=self.sigma[i])
            ll += np.log(p)
        return ll / self.N
    
    # 运行GMM算法的E步骤、M步骤迭代循环
    def fit(self, X):
        self.N = len(X)
        self.Y = np.zeros((self.N, self.K))
        ll = self.log_likelihood(X)
        print('开始迭代')
        for i in range(self.max_iter):
            self.E_step(X)
            self.M_step(X)
            new_ll = self.log_likelihood(X)
            print(f'第{i}步, log-likelihood = {new_ll:.4f}')
            if new_ll - ll < 1e-4:
                print('log-likelihood不再变化，退出程序')
                break
            else:
                ll = new_ll
    
    # 根据学习到的参数将一个数据点分配到概率最大的簇
    def transform(self, X):
        assert hasattr(self, 'Y') and len(self.Y) == len(X)
        return np.argmax(self.Y, axis=1)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

与*k*均值聚类方法类似，在使用最大期望值法的高斯混合的情况下，观察在Books数据集3个真实类别中不同簇的占比：

In [13]:
# 直接对TF-IDF特征聚类运行速度过慢，因此使用PCA降维，将TF-IDF向量降到50维
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
train_P = pca.fit_transform(train_F)

# 运行GMM算法，展示聚类结果
gmm = GMM(K, dim=train_P.shape[1])
clusters = gmm.fit_transform(train_P)
print(clusters)
show_clusters(clusters, K)

开始迭代
第0步, log-likelihood = 77.2685
...
第10步, log-likelihood = 95.9564
...
第20步, log-likelihood = 97.8945
...
第30步, log-likelihood = 98.2401
...
第39步, log-likelihood = 98.2509
log-likelihood不再变化，退出程序
[2 0 2 ... 1 2 1]
计算机类:	{ 0: 114(0.03), 1: 1256(0.33), 2: 2472(0.64), }
艺术传媒类:	{ 0: 2129(0.93), 1: 23(0.01), 2: 148(0.06), }
经管类:	{ 0: 268(0.11), 1: 2152(0.87), 2: 65(0.03), }


下面演示基于朴素贝叶斯模型的聚类算法实现：

In [14]:
from scipy.special import logsumexp

# 无监督朴素贝叶斯
class UnsupervisedNaiveBayes:
    def __init__(self, K, dim, max_iter=100):
        self.K = K
        self.dim = dim
        self.max_iter = max_iter
        
        # 初始化参数，pi为先验概率分布，P用于保存K个朴素贝叶斯模型的参数
        self.pi = np.ones(K) / K
        self.P = np.random.random((K, dim))
        self.P /= self.P.sum(axis=1, keepdims=True)
        
    # E步骤
    def E_step(self, X):
        # 根据朴素贝叶斯公式，计算每个数据点分配到每个簇的概率分布
        for i, x in enumerate(X):
            # 由于朴素贝叶斯使用了许多概率连乘，容易导致精度溢出，
            # 因此使用对数概率
            self.Y[i, :] = np.log(self.pi) + (np.log(self.P) *\
                x).sum(axis=1)
            # 使用对数概率、logsumexp和exp，等价于直接计算概率，
            # 好处是数值更加稳定
            self.Y[i, :] -= logsumexp(self.Y[i, :])
            self.Y[i, :] = np.exp(self.Y[i, :])
    
    # M步骤
    def M_step(self, X):
        # 根据估计的簇概率分布更新先验概率分布
        self.pi = self.Y.sum(axis=0) / self.N
        self.pi /= self.pi.sum()
        # 更新每个朴素贝叶斯模型的参数
        for i in range(self.K):
            self.P[i] = (self.Y[:, i:i+1] * X).sum(axis=0) / \
                (self.Y[:, i] * X.sum(axis=1)).sum()
        # 防止除0
        self.P += 1e-10
        self.P /= self.P.sum(axis=1, keepdims=True)
    
    # 计算对数似然，用于判断迭代终止
    def log_likelihood(self, X):
        ll = 0
        for x in X:
            # 使用对数概率和logsumexp防止精度溢出
            logp = []
            for i in range(self.K):
                logp.append(np.log(self.pi[i]) + (np.log(self.P[i]) *\
                    x).sum())
            ll += logsumexp(logp)
        return ll / len(X)
    
    # 无监督朴素贝叶斯的迭代循环
    def fit(self, X):
        self.N = len(X)
        self.Y = np.zeros((self.N, self.K))
        ll = self.log_likelihood(X)
        print(f'初始化log-likelihood = {ll:.4f}')
        print('开始迭代')
        for i in range(self.max_iter):
            self.E_step(X)
            self.M_step(X)
            new_ll = self.log_likelihood(X)
            print(f'第{i}步, log-likelihood = {new_ll:.4f}')
            if new_ll - ll < 1e-4:
                print('log-likelihood不再变化，退出程序')
                break
            else:
                ll = new_ll
    
    def transform(self, X):
        assert hasattr(self, 'Y') and len(self.Y) == len(X)
        return np.argmax(self.Y, axis=1)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

In [15]:
# 根据朴素贝叶斯模型，需要统计出每个数据点包含的词表中每个词的数目
train_C = np.zeros((len(train_X), vocab_size))
for i, data in enumerate(train_X):
    for token_id in data:
        train_C[i, token_id] += 1

unb = UnsupervisedNaiveBayes(K, dim=vocab_size, max_iter=100)
clusters = unb.fit_transform(train_C)
print(clusters)
show_clusters(clusters, K)

初始化log-likelihood = -779.0355
开始迭代
第0步, log-likelihood = -589.0541
...
第10步, log-likelihood = -571.5391
...
第20步, log-likelihood = -567.4288
...
第30步, log-likelihood = -567.3908
...
第38步, log-likelihood = -567.3578
log-likelihood不再变化，退出程序
[1 2 1 ... 1 1 1]
计算机类:	{ 0: 307(0.08), 1: 3437(0.89), 2: 98(0.03), }
艺术传媒类:	{ 0: 59(0.03), 1: 156(0.07), 2: 2085(0.91), }
经管类:	{ 0: 2252(0.91), 1: 79(0.03), 2: 154(0.06), }
